In [0]:
%sql  
/* gold_feedback_metrics.ipynb

GOAL 1: Satisfaction score metrics (avg, counts, comment stats) by specialty & source  
GOAL 2: Tag usage and average satisfaction per tag  
GOAL 3: Encounter–feedback match rate  

SOURCE: kardia_silver.silver_feedback_enriched  
OUTPUT:  
  TABLE: gold_feedback_satisfaction
  TABLE: gold_feedback_tag_analysis
  TABLE: gold_feedback_encounter_match

TRIGGER: Full snapshot overwrite each run — simple for small datasets. */

-- Ensure Gold GB exists
CREATE DATABASE IF NOT EXISTS kardia_gold;
USE kardia_gold;

In [0]:
%sql
-- 2. Enhanced satisfaction metrics by provider_specialty & source
CREATE OR REPLACE TABLE gold_feedback_satisfaction AS
SELECT
  COALESCE(provider_specialty, 'Unknown') AS provider_specialty,
  COALESCE(provider_id, 'Unknown')       AS provider_id,
  COALESCE(source, 'Unknown')             AS source,
  COUNT(*)                                AS feedback_count,
  ROUND(AVG(satisfaction_score), 2)       AS avg_score,
  ROUND(STDDEV_POP(satisfaction_score), 2) AS stddev_score,
  PERCENTILE_APPROX(satisfaction_score, 0.5) AS median_score,
  SUM(CASE WHEN comments IS NOT NULL THEN 1 ELSE 0 END) AS comments_count,
  ROUND(100.0 * SUM(CASE WHEN comments IS NOT NULL THEN 1 ELSE 0 END) / COUNT(*), 2) AS pct_with_comments,
  ROUND(AVG(LENGTH(comments)), 2)         AS avg_comment_length
FROM kardia_silver.silver_feedback_enriched
GROUP BY COALESCE(provider_specialty, 'Unknown'),
         COALESCE(provider_id, 'Unknown'),
         COALESCE(source, 'Unknown');

In [0]:
%sql  
-- 3. Tag usage and average satisfaction per tag  
CREATE OR REPLACE TABLE gold_feedback_tag_analysis AS
WITH exploded AS (
  SELECT satisfaction_score, EXPLODE(tags) AS tag
  FROM   kardia_silver.silver_feedback_enriched
)
SELECT
  tag,
  COUNT(*) AS feedback_count,
  ROUND(AVG(satisfaction_score), 2) AS avg_score
FROM exploded
GROUP BY tag
ORDER BY feedback_count DESC;

In [0]:
%sql  
-- 4. Encounter–feedback match rate (has valid visit_id?)  
CREATE OR REPLACE TABLE gold_feedback_encounter_match AS
SELECT
  COUNT(*) AS total_feedback_submissions,
  SUM(CASE WHEN visit_id IS NOT NULL THEN 1 ELSE 0 END) AS matched_to_encounter_count,
  ROUND(100.0 * SUM(CASE WHEN visit_id IS NOT NULL THEN 1 ELSE 0 END) / COUNT(*), 2) AS match_rate_percent
FROM kardia_silver.silver_feedback_enriched;

In [0]:
%sql  
-- 5. Preview: satisfaction metrics  
SELECT *  
FROM gold_feedback_satisfaction
ORDER BY provider_specialty, source  
LIMIT 20;

In [0]:
%sql  
-- Preview: tag analysis  
SELECT * FROM gold_feedback_tag_analysis LIMIT 10;

In [0]:
%sql  
-- Preview: encounter–feedback match rate  
SELECT * FROM gold_feedback_encounter_match LIMIT 10;